In [2]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool, Process
import os


In [2]:
def getAllUrl(url, url_base, urls, errors):
    
    page = requests.get(url)

    if page.status_code == 200:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        for anchor in soup.find_all("a"):
            if "href" in anchor.attrs:
                if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                    absolute_url == url+"?pR="+anchor["page"]
                # Manque "?pI=" pour les interviews
                absolute_url = urljoin(url_base, anchor["href"])
                if "download" not in absolute_url and absolute_url not in urls:
                    if re.search(url_base, absolute_url):
                        urls.append(absolute_url)

        # Export des pages html ?

    else:
        print(" ", page.status_code)
        errors.append(url)



In [3]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [127]:
def get_all_page(urls, pattern):

    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]

    iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
    for item in iter:
        lst = re.findall("url.*?;", item.find_next_sibling().text)
        nw_url = ""
        if len(lst) > 0:
            for item in lst:
                if re.search("querystring", item):
                    continue
                else:
                    item = re.sub("url =|url \+=|url", "", item)
                    item = item.replace(";", "").replace("\"", "").strip()
                    item = item.split(" + ")
                for part in item:
                    nw_url += part
            match = re.compile("\$\(this\)\.attr\(page\)")
            for i in range(1, int(last_page)+1):
                urls.append(re.sub(match,str(i), nw_url))
        
        else:
            for i in range(1, int(last_page)+1):
                urls.append(re.sub("page=.", f"page={i}", pattern))
    
    return(urls)

In [6]:

EXPORT_FILE = "M3urls3.txt"

if __name__ == "__main__":
    
    url_base = 'https://www.millenaire3.com'
    errors = []
    urls = get_all_page(urls = [], pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=")
    # if EXPORT_FILE in os.listdir():
    #     urls = read_urls(EXPORT_FILE)
    # else:
    #     urls = [url_base]

    i = 0# On peut changer la valeur de départ
    while i <= len(urls)-1:
        if re.search("/partage_email/|/generated_pdf/", urls[i]):
            errors.append(urls[i])
            print(f"\r{i} / {len(urls)}")
        else:
            getAllUrl(urls[i], url_base, urls, errors)
            i += 1
            print(f"\r{i} / {len(urls)} : {urls[i-1]}", end = "")

with open(EXPORT_FILE, "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url+"\n")

363
2507 / 10277 : https://www.millenaire3.com/ressources/conception-eco-culturelle-d-un-centre-de-rechercheuesetudeepreuve-des-exigences-renouvelees-des-usagers-en-francenpied-et-les-deux-roues-soient-plus-efficaces1%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=  404
4016 / 13523 : https://www.millenaire3.com/toutes-les-videosgeneric-vie-j-habitentreelisme-de-l-economie-au-caeur-des-enjeux-deja-la-generation-zes-precairesyonnaiseuees-de-brevets-et-d-innovationsaeslyon-et-turin

In [ ]:
print(errors[0])

In [34]:
urls = read_urls(EXPORT_FILE)

12367


In [175]:
# pattern = "https://www.millenaire3.com/tous-les-auteurs?p=1&l=B&&l=B&&l=B&l=B"
# html = requests.get(pattern)
# soup = BeautifulSoup(html.content, "html.parser")
# last_page = soup.find("a", {"title" : "Dernière page"})["page"]
# last_page

# item = soup.find("div", {"class" : "m3-pagination print_hidden"})
# print(item.find_next_sibling().text)
# lst = re.findall("url.*?;", item.find_next_sibling().text)
# if lst != None:
#     for item in lst:
#         if "$(this).attr" in item : 
#             print(item.split(" ")[2])

url = "https://www.millenaire3.com/tous-les-auteurs?p=1&l=B&&l=B&&l=B&l=B"
url2 = "https://www.millenaire3.com/auteurs/piessat-lucas"
url3 = "https://www.millenaire3.com/tag/cohesion-sociale"
url4 = "https://www.millenaire3.com/auteurs/vievard-ludovic"

pattern = url3

html = requests.get(pattern)
soup = BeautifulSoup(html.content, "html.parser")
last_page = soup.find("a", {"title" : "Dernière page"})["page"]

iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
for item in iter:
    lst = re.findall("url.*?;", item.find_next_sibling().text)
    nw_url = ""
    if len(lst) > 0:
        for item in lst:
            if re.search("querystring", item):
                continue
            else:
                item = re.sub("url =|url \+=|url", "", item)
                item = item.replace(";", "").replace("\"", "").strip()
                item = item.split(" + ")
            for part in item:
                nw_url += part
        match = re.compile("\$\(this\)\.attr\(page\)")
        for i in range(1, int(last_page)+1):
            print(re.sub(match,str(i), nw_url))

/tag/cohesion-sociale?pr=1
/tag/cohesion-sociale?pr=2
/tag/cohesion-sociale?pr=3
/tag/cohesion-sociale?pr=4
/tag/cohesion-sociale?pr=5
/tag/cohesion-sociale?pr=6
/tag/cohesion-sociale?pr=7
/tag/cohesion-sociale?pr=8


In [84]:
"https://www.millenaire3.com/tous-les-auteurs?p=1&l=B&&l=B&&l=B&l=B"[re.search("\?", "https://www.millenaire3.com/tous-les-auteurs?p=1&l=B&&l=B&&l=B&l=B").span()[0]]

'?'